In [5]:
import os
import pandas as pd
import numpy as np
from utils import seed_torch
import argparse
import train_configs
def split_kfold(input_csv,output_csv,n_fold=5,n_class_skip=1):
    train = pd.read_csv(input_csv)
    train.columns = ['image','Id']
    train = train[train['Id']!='new_whale']
    train['classes'] = train.groupby('Id').ngroup()
    train['classes_species'] = 0
    df_list = []
    for index,df in train.groupby('classes'):
        if len(df)>n_class_skip:
            df = df.sample(frac=1)
            df['fold'] = (np.arange(len(df)) + np.random.randint(n_fold)) % n_fold
            df_list.append(df)
        else:
            df['fold'] = -1
            df_list.append(df) 
    train = pd.concat(df_list)
    for row in range(n_fold):
        print(len(list(set(train[train.fold==row].classes_species))))
    df = train
    df_individual_id = df.groupby('Id').count()[['image']]
    df_individual_id.columns = ['count_individual_id']
    df = df.merge(df_individual_id,on='Id')
    df.to_csv('/home/kaor/whale_code/csv/fold2.csv',index=False)
    df = df[df.count_individual_id>0]
    df.to_csv(output_csv,index=False)


if __name__ == "__main__":
    config = eval(f"train_configs.config16pretrain.config")
    seed_torch(config.seed)
    split_kfold(config.input_csv,config.kfold_csv,config.n_folds,config.n_class_skip)

/home/kaor/.conda/envs/openmmlab/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1
1
1
1
1


In [6]:
len(list(set(pd.read_csv(config.kfold_csv)['classes'])))

5004

In [7]:
pd.read_csv(config.kfold_csv)

,image,Id,classes,classes_species,fold,count_individual_id
0,833675975.jpg,w_0003639,0,0,-1,1
1,2fe2cc5c0.jpg,w_0003c59,1,0,-1,1
2,cc6c1a235.jpg,w_0027efa,2,0,2,10
3,30eac8c9f.jpg,w_0027efa,2,0,3,10
4,7b9a5e4d8.jpg,w_0027efa,2,0,4,10
...,...,...,...,...,...,...
15692,4f9e431aa.jpg,w_ffa542b,5000,0,1,3
15693,8abea550e.jpg,w_ffa542b,5000,0,2,3
15694,9dd320bb3.jpg,w_ffa6d42,5001,0,-1,1
15695,d3967e1df.jpg,w_ffcf5fe,5002,0,-1,1
